In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns

import string
import sqlite3

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
from sklearn.metrics import confusion_matrix, auc, roc_curve
from nltk.stem.porter import PorterStemmer

%matplotlib inline

In [8]:
conn = sqlite3.connect("datasets/food_reviews.sqlite")

In [18]:
filtered_data = pd.read_sql_query("""
    select * from Reviews
    WHERE Score != 3
""",conn)

In [19]:
actualScores = filtered_data['Score']
posNeg = actualScores.map(lambda i:"Positive" if i>3 else "Negative")
filtered_data["Score"] = posNeg

In [20]:
print(filtered_data.shape)
filtered_data.head(3)

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


## Data Cleaning : DeDuplication